In [ ]:
from IPython.display import clear_output

In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
clear_output()

In [ ]:
!pip install transformers sentencepiece 
clear_output()

In [ ]:
# !pip uninstall torch
# !pip inst

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch_xla
from keras.preprocessing.sequence import pad_sequences
import torch_xla.core.xla_model as xm
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig, get_linear_schedule_with_warmup, AutoModelForQuestionAnswering, AutoModel, AutoConfig, AutoTokenizer
from tqdm import tqdm, trange
import pandas as pd
import io
import torch_xla.distributed.xla_multiprocessing as xmp
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt
import torch_xla.distributed.parallel_loader as pl
import torch_xla.debug.metrics as met
import os
os.environ['XLA_USE_BF16']="1"
clear_output()

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/nti/rucos/

[Errno 2] No such file or directory: 'drive/MyDrive/nti/rucos/'
/content/drive/MyDrive/nti/rucos


In [ ]:
!ls

pytorch-xla-env-setup.py
sbersquad_train.json
test.jsonl
torch-1.6-cp37-cp37m-linux_x86_64.whl
torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torch-nightly-cp37-cp37m-linux_x86_64.whl
torchvision-1.6-cp37-cp37m-linux_x86_64.whl
torchvision-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torchvision-nightly-cp37-cp37m-linux_x86_64.whl
torch_xla-1.6-cp37-cp37m-linux_x86_64.whl
torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torch_xla-nightly-cp37-cp37m-linux_x86_64.whl
train.jsonl
val.jsonl


In [ ]:
import json

with open('train.jsonl', 'r', encoding='utf-8') as data:
  dat = list(data)
ex = json.loads(dat[6])
# for el in ex['passage']:
#   print(el)

print(f"text : {ex['passage']['text']}")
print(f"entities : {ex['passage']['entities']}")
print(f"qas : {ex['qas']}")
print(ex['qas'][0]['query'])
print(ex)

text : Требование и.о. чеченского премьера прекратить деятельность Датского совета помощи беженцам расценивается немецкими журналистами как политическая и пиар-акция, не имеющая ничего общего с конфликтом вокруг карикатур. Через несколько дней после начала массовых демонстраций протеста в мусульманских странах, вызванных публикацией в ряде западноевропейских газет карикатур на пророка Мухаммеда, и.о. премьер-министра Чечни Рамзан Кадыров фактически потребовал объявить работающих в республике сотрудников Датский совет по беженцам персонами "нон грата". Антидатская демонстрация в Индонезии
Указанная им причина – Чечня присоединяется к "бойкоту всего датского", который объявлен в ряде исламских стран. Однако если, например, в Саудовской Аравии речь идет прежде всего о товарах датского производства, то руководство Чечне обратило свои санкции против граждан Дании, оказывающих гуманитарную помощь. Концентрация власти или вводная сверху
Размышляя о заявлении Кадырова, которое он сделал 6 февр

In [ ]:
print

<function print>

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("deepset/xlm-roberta-large-squad2")

In [ ]:
train_text = []
train_query = []
answers = []
for example in dat[:20000]:
  ex = json.loads(example)
  text = ex['passage']['text']
  query = ex['qas'][0]['query']
  # length.append(len(tokenizer.tokenize(text)))
  for i in ex['qas'][0]['answers']:
    answers.append([i['start'], i['end']])
    train_text.append(text[:256])
    train_query.append(query)

In [ ]:
with open('val.jsonl', 'r', encoding='utf-8') as data:
  dat = list(data)

val_text = []
val_query = []
val_answers = []
for example in dat[:20000]:
  ex = json.loads(example)
  text = ex['passage']['text']
  query = ex['qas'][0]['query']
  # length.append(len(tokenizer.tokenize(text)))
  for i in ex['qas'][0]['answers']:
    val_answers.append([i['start'], i['end']])
    val_text.append(text[:256])
    val_query.append(query)

In [ ]:
pad_on_right = True

In [ ]:
def prepare_train_features(questions, texts, labels):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
  tokenized_examples = tokenizer(questions, texts, 
                              max_length=450,
                              padding='max_length',
                              stride=128,
                              truncation=True,
                              return_overflowing_tokens=True,
                              return_offsets_mapping=True
                              )


  sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

  offset_mapping = tokenized_examples.pop("offset_mapping")

  tokenized_examples["start_positions"] = []
  tokenized_examples["end_positions"] = []

  for i, offsets in enumerate(offset_mapping):
      # We will label impossible answers with the index of the CLS token.
      input_ids = tokenized_examples["input_ids"][i]
      cls_index = input_ids.index(tokenizer.cls_token_id)

      # Grab the sequence corresponding to that example (to know what is the context and what is the question).
      sequence_ids = tokenized_examples.sequence_ids(i)

      # One example can give several spans, this is the index of the example containing this span of text.
      sample_index = sample_mapping[i]
      answers = labels[sample_index]
      # If no answers are given, set the cls_index as answer.
      if len(answers) == 0:
          tokenized_examples["start_positions"].append(cls_index)
          tokenized_examples["end_positions"].append(cls_index)
      else:
          # Start/end character index of the answer in the text.
          start_char = answers[0]
          end_char = answers[1]

          # Start token index of the current span in the text.
          token_start_index = 0
          while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
              token_start_index += 1

          # End token index of the current span in the text.
          token_end_index = len(input_ids) - 1
          while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
              token_end_index -= 1

          # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
          if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
              tokenized_examples["start_positions"].append(cls_index)
              tokenized_examples["end_positions"].append(cls_index)
          else:
              # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
              # Note: we could go after the last offset if the answer is the last word (edge case).
              while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                  token_start_index += 1
              tokenized_examples["start_positions"].append(token_start_index - 1)
              while offsets[token_end_index][1] >= end_char:
                  token_end_index -= 1
              tokenized_examples["end_positions"].append(token_end_index + 1)

  return tokenized_examples
train_data = prepare_train_features(questions=train_query, texts=train_text, labels=answers)
validation_data = prepare_train_features(questions=val_text,texts=val_text, labels=val_answers)

In [ ]:
FLAGS = {}
# FLAGS['data_dir'] = "/tmp/cifar"
# FLAGS['batch_size'] = 4
FLAGS['num_workers'] = 8
FLAGS['learning_rate'] = 2e-5
FLAGS['momentum'] = 0.9
FLAGS['num_epochs'] = 8
FLAGS['num_cores'] = 8 if os.environ.get('TPU_NAME', None) else 1
FLAGS['log_steps'] = 1
FLAGS['metrics_debug'] = False

In [ ]:
print(os.environ.get(('TPU_NAME'), None))

grpc://10.99.144.82:8470


In [ ]:
!ls

pytorch-xla-env-setup.py
sbersquad_train.json
test.jsonl
torch-1.6-cp37-cp37m-linux_x86_64.whl
torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torch-nightly-cp37-cp37m-linux_x86_64.whl
torchvision-1.6-cp37-cp37m-linux_x86_64.whl
torchvision-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torchvision-nightly-cp37-cp37m-linux_x86_64.whl
torch_xla-1.6-cp37-cp37m-linux_x86_64.whl
torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torch_xla-nightly-cp37-cp37m-linux_x86_64.whl
train.jsonl
val.jsonl


# Sber(old)

In [ ]:
# import transformers
# from transformers import XLMRobertaTokenizerFast
# tokenizer = XLMRobertaTokenizerFast.from_pretrained('roberta-base')
# assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)
# tokenizer.padding_side = "right"
# assert len(question_train) == len(text_train)
# pad_on_right = True

In [ ]:
#@title prepare train feautures
# def prepare_train_features(questions, texts):
#     # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
#     # in one example possible giving several features when a context is long, each of those features having a
#     # context that overlaps a bit the context of the previous feature.
#   tokenized_examples = tokenizer(questions, texts, 
#                               max_length=512,
#                               padding='max_length',
#                               stride=128,
#                               truncation=True,
#                               return_overflowing_tokens=True,
#                               return_offsets_mapping=True
#                               )


#   sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

#   offset_mapping = tokenized_examples.pop("offset_mapping")

#   tokenized_examples["start_positions"] = []
#   tokenized_examples["end_positions"] = []

#   for i, offsets in enumerate(offset_mapping):
#       # We will label impossible answers with the index of the CLS token.
#       input_ids = tokenized_examples["input_ids"][i]
#       cls_index = input_ids.index(tokenizer.cls_token_id)

#       # Grab the sequence corresponding to that example (to know what is the context and what is the question).
#       sequence_ids = tokenized_examples.sequence_ids(i)

#       # One example can give several spans, this is the index of the example containing this span of text.
#       sample_index = sample_mapping[i]
#       answers = labels[sample_index]
#       # If no answers are given, set the cls_index as answer.
#       if len(answers) == 0:
#           tokenized_examples["start_positions"].append(cls_index)
#           tokenized_examples["end_positions"].append(cls_index)
#       else:
#           # Start/end character index of the answer in the text.
#           start_char = answers[0]
#           end_char = answers[1]

#           # Start token index of the current span in the text.
#           token_start_index = 0
#           while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
#               token_start_index += 1

#           # End token index of the current span in the text.
#           token_end_index = len(input_ids) - 1
#           while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
#               token_end_index -= 1

#           # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
#           if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
#               tokenized_examples["start_positions"].append(cls_index)
#               tokenized_examples["end_positions"].append(cls_index)
#           else:
#               # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
#               # Note: we could go after the last offset if the answer is the last word (edge case).
#               while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
#                   token_start_index += 1
#               tokenized_examples["start_positions"].append(token_start_index - 1)
#               while offsets[token_end_index][1] >= end_char:
#                   token_end_index -= 1
#               tokenized_examples["end_positions"].append(token_end_index + 1)

#   return tokenized_examples
# train_data = prepare_train_features(question_train[:10000], text_train[:10000])
# validation_data = prepare_train_features(question_train[11000:12001],text_train[11000:12001])

# ну раздел и раздел


In [ ]:
from transformers import AdamW, BertForSequenceClassification
from torch import nn
from sklearn.metrics import f1_score

NUM_EPOCHS = 8
BATCH = 1

In [ ]:
def preprocessing(train_inputs, validation_data,
                  batch_size):
    # train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(
    #     input_ids, train_gt_orig, 
    #     random_state=42,
    #     test_size=test_SIZE
    # )

    train_in = torch.tensor(train_inputs['input_ids'], dtype=torch.long)
    train_mask = torch.tensor(train_inputs['attention_mask'])
    # train_labels = torch.tensor(train_inputs['start_score'], dtype=torch.long)
    start_score = torch.tensor(train_inputs['start_positions'], dtype=torch.long)
    end_score = torch.tensor(train_inputs['end_positions'], dtype=torch.long)
    training_data = TensorDataset(train_in, train_mask,start_score, end_score)



    validation_inputs = torch.tensor(validation_data['input_ids'], dtype=torch.long)
    validation_mask = torch.tensor(validation_data['attention_mask'], dtype=torch.long)
    validation_start = torch.tensor(validation_data['start_positions'], dtype=torch.long)
    validation_end = torch.tensor(validation_data['end_positions'], dtype=torch.long)
    # valid_labels = torch.tensor(valid_labels, dtype=torch.long)
    valid_data = TensorDataset(validation_inputs, validation_start,validation_start, validation_end)

    

    train_sampler =  torch.utils.data.distributed.DistributedSampler(   #SequentialSampler(train_data)
      training_data,
      num_replicas=xm.xrt_world_size(),
      rank=xm.get_ordinal(),
      shuffle=False)
    train_dataloader = DataLoader(
        training_data,
        sampler=train_sampler,
        batch_size=batch_size,
        num_workers = 0
    )
    
    
    val_sampler = torch.utils.data.distributed.DistributedSampler(     #SequentialSampler(valid_data)
      valid_data,
      num_replicas=xm.xrt_world_size(),
      rank=xm.get_ordinal(),
      shuffle=False) 
    validation_dataloader = DataLoader(
        valid_data,
        sampler=val_sampler,
        batch_size=batch_size,
        num_workers = 0
    )
    
    
    return train_dataloader, validation_dataloader

In [ ]:
train_dataloader, validation_dataloader = preprocessing(train_inputs=train_data,
                                                          validation_data=validation_data,
                                                      batch_size=BATCH)

In [ ]:
# class DBQA(torch.nn.Module):
#     def __init__(self, m):
#         super(DBQA, self).__init__()    
#         self.model= m
#         self.d1 = torch.nn.Dropout(0.3)
#         self.l1 = torch.nn.Linear(768, 2)
#         self.model.apply_state()

#     def forward(self, input_ids, attention_mask):
#         _, x  = self.model(input_ids=input_ids, attention_mask=attention_mask)
#         x = self.d1(x)
#         x = self.l1(x)
#         return x  
# m = AutoModel.from_pretrained("google/mt5-large")

In [ ]:
train_loss_set = []
import time
def model_training():
  # global train_loss_set

  # del inputs_train, inputs_valid
  torch.manual_seed(42)
  device = xm.xla_device()
  model = AutoModelForQuestionAnswering.from_pretrained("deepset/xlm-roberta-large-squad2")
  clear_output()
  # devices = (
  #   xm.get_xla_supported_devices(
  #       max_devices=FLAGS['num_cores']) if FLAGS['num_cores'] != 0 else [])
  
  # model = xmp.MpModelWrapper(m)
  model = model.to(device)
  
  
  lr = 2e-5 * 8

  optimizer = AdamW(model.parameters(), lr=lr)


  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=len(train_dataloader), 
                                            num_training_steps=len(train_dataloader)*NUM_EPOCHS)
  
  
  def train(loader):                     # train_loss_set = []
      model.train()                                     # train_loss = 0
      tracker = xm.RateTracker()
      for step, batch in enumerate(loader):
          batch = tuple(t.to(device) for t in batch)
          input_ids, mask, start, end = batch
          optimizer.zero_grad()
          loss = model(input_ids,mask, start, end)
          # train_loss_set.append(loss[0].item())  
          loss[0].backward()
          xm.optimizer_step(optimizer)
          scheduler.step()
          # clear_output()
          tracker.add(BATCH)
          # if step % 100 == 0:
          print('[xla:{}]({}/{}) Loss={:.5f} Rate={:.2f} GlobalRate={:.2f} Time={}'.format(
                xm.get_ordinal(), step,len(train_dataloader), loss[0].item(), tracker.rate(),
                tracker.global_rate(), time.asctime()[10:-5]), flush=True)

      
    


  def evaluation(dataloader):
      global start_logits, end_logits
      model.eval()
      valid_labels, valid_preds = [], []
      for batch in dataloader:   
          batch = tuple(t.to(device) for t in batch)
          
          input_ids, mask, start, end= batch
          with torch.no_grad():
              logits =  model(input_ids,mask, start, end)
          # logits = logits[0].
          preds = logits[0].to('cpu').numpy()
          batch_preds = np.argmax(preds, axis=1)
          batch_labels = np.stack(labels, axis=0) 
          valid_preds.append(batch_preds) 
          batch_acc = accuracy_score(valid_preds, valid_labels)
          batch_f1 = f1_score(valid_preds, valid_labels )
          plt.sublpot(batch_acc)
          plt.ylabel('accuracy')
          plt.show()
          plt.subplot(batch_f1)
          plt.ylabel('f1')
          plt.show()
      return valid_preds
    
  for epoch in range(1, NUM_EPOCHS + 1):
      para_loader = pl.ParallelLoader(train_dataloader, [device])
      train(para_loader.per_device_loader(device))
      xm.master_print("---------------------------------------------- Finished training epoch {} -------------------------------".format(epoch))

      para_loader = pl.ParallelLoader(validation_dataloader, [device])
      pred, target  = evaluation(para_loader.per_device_loader(device))
  
  
  
  return valid_preds

In [ ]:
!cat /proc/cpuinfo | grep processor | wc -l
!free -h

2
              total        used        free      shared  buff/cache   available
Mem:            12G         10G        207M        2.2M        2.4G        6.0G
Swap:            0B          0B          0B


In [ ]:
def _mp_fn(rank, flags):
  global FLAGS
  torch.set_default_tensor_type('torch.FloatTensor')
  preds = model_training()
  FLAGS = flags

xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

ProcessExitedException: ignored

In [ ]:
def postprocess_qa_predictions(examples, features, start, end, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = start_logits, end_logits
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions